Setup environment and load the base PyPSA-Earth network for a specified country.


In [1]:
# --- Core imports
import os
import sys
import copy
import logging
import warnings
from os.path import join
from pathlib import Path

# --- Numerical / data
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

# --- Power systems / geospatial
import pypsa
import atlite
from shapely.geometry import Point, box
from shapely.ops import unary_union
import shapely

# Optional plotting (only used in helper plot function)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Optional: powerplantmatching (picks up config but we keep local CSV as source of truth)
try:
    import powerplantmatching as pm
    HAVE_PPM = True
except Exception:
    HAVE_PPM = False

# --- Silence noisy warnings
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.filterwarnings('ignore')


# --- Logging
parent_dir = Path(os.getcwd()).parents[0]          # project/
LOG_FILE = join(parent_dir, "logs.log")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.FileHandler(LOG_FILE, encoding="utf-8"), logging.StreamHandler(sys.stdout)],
)
logger = logging.getLogger(__name__)

# --- Project paths helper
sys.path.append(str(parent_dir))
from src.paths import all_dirs  # must exist in your repo
dirs = all_dirs()

# --- Add PyPSA-Earth scripts to PATH (assumes repo layout: <project>/../pypsa-earth/scripts)
scripts_path = os.path.join(parent_dir.parents[0], "pypsa-earth", "scripts")
assert os.path.isdir(scripts_path), f"Path not found: {scripts_path}"
sys.path.append(scripts_path)


In [2]:
# 1) Load Networks
network_dir = dirs["data/processed/networks"]
network_files = [
    "network_original",
    "network_snapped",
    "network_expanded",
    "network_expanded_no_orphans",
    "network_nuclear",
    "network_prod_mix",
    "network_base",
]

logger.info("Loading .nc networks …")
networks_dict = {}
for nf in network_files:
    f = join(network_dir, f"{nf}.nc")
    if os.path.isfile(f):
        try:
            networks_dict[nf] = pypsa.Network(f)
            logger.info(f"  ✓ loaded {nf}.nc")
        except Exception as e:
            logger.warning(f"  ⚠ failed to load {nf}.nc: {e}")
    else:
        logger.warning(f"  ⚠ missing file: {f}")

# Work on a copy of the base network
assert "network_base" in networks_dict, "network_base.nc not found/loaded."
network = networks_dict["network_base"].copy()


2025-11-11 11:00:59: Loading .nc networks …
2025-11-11 11:00:59: Imported network network_original.nc has buses, lines, transformers
2025-11-11 11:00:59:   ✓ loaded network_original.nc
2025-11-11 11:00:59: Imported network network_snapped.nc has buses, lines, transformers
2025-11-11 11:00:59:   ✓ loaded network_snapped.nc
2025-11-11 11:00:59: Imported network network_expanded.nc has buses, lines, transformers
2025-11-11 11:00:59:   ✓ loaded network_expanded.nc
2025-11-11 11:00:59: Imported network network_expanded_no_orphans.nc has buses, lines, transformers
2025-11-11 11:00:59:   ✓ loaded network_expanded_no_orphans.nc
2025-11-11 11:01:00: Imported network network_nuclear.nc has buses, lines, transformers
2025-11-11 11:01:00:   ✓ loaded network_nuclear.nc
2025-11-11 11:01:00: Imported network network_prod_mix.nc has buses, lines, transformers
2025-11-11 11:01:00:   ✓ loaded network_prod_mix.nc
2025-11-11 11:01:00: Imported network network_base.nc has buses, lines, transformers
2025-11

In [3]:
# 2) Loads: add p_set time series
# 2) Loads: add p_set time series with nearest-bus remapping for unknown buses
try:
    path_loads = dirs["data/processed/scaled_loads"]
    load_profile_name = "load_base_2030_linear.csv"
    load_profile_file = join(path_loads, load_profile_name)
    logger.info(f"Loading load profile: {load_profile_file}")
    load_profile = pd.read_csv(load_profile_file, index_col=0, parse_dates=True)

    # --- Ensure column names are strings
    load_cols = [str(c) for c in load_profile.columns]
    load_profile.columns = load_cols

    # --- Existing buses in the active network
    buses_now = network.buses.copy()
    buses_now.index = buses_now.index.astype(str)
    existing_buses = set(buses_now.index)

    # --- Find missing bus labels
    missing = [c for c in load_cols if c not in existing_buses]
    if missing:
        logger.warning(f"{len(missing)} load buses not found; reassigning to nearest existing bus.")
    else:
        logger.info("All load buses found in current network.")

    # --- Build KDTree on current network buses
    from scipy.spatial import cKDTree as KDTree
    use_lonlat = {"lon", "lat"}.issubset(buses_now.columns)
    if use_lonlat:
        coords_now = buses_now.loc[:, ["lon", "lat"]].to_numpy(dtype=float)
    else:
        coords_now = buses_now.loc[:, ["x", "y"]].to_numpy(dtype=float)
    kdt_now = KDTree(coords_now)

    # --- Helper: try to get coordinates for a missing bus from any fallback network
    def find_bus_coords_in_fallbacks(bus_name: str):
        for nf, net_fb in networks_dict.items():
            try:
                df = net_fb.buses.copy()
                df.index = df.index.astype(str)
                if bus_name in df.index:
                    if {"lon", "lat"}.issubset(df.columns):
                        return ("lonlat",
                                float(df.loc[bus_name, "lon"]),
                                float(df.loc[bus_name, "lat"]))
                    elif {"x", "y"}.issubset(df.columns):
                        return ("xy",
                                float(df.loc[bus_name, "x"]),
                                float(df.loc[bus_name, "y"]))
            except Exception:
                pass
        return None

    # --- Build a bus assignment vector aligned with load columns
    bus_assignments = []
    remap_report = []

    for c in load_cols:
        if c in existing_buses:
            bus_assignments.append(c)  # unchanged
            continue

        # Try to recover coordinates for this bus label from other networks
        info = find_bus_coords_in_fallbacks(c)
        if info is not None:
            kind, bx, by = info
            # Query KDTree (always built for current network in the coordinate type we have)
            if use_lonlat and kind == "lonlat":
                q = np.array([[bx, by]])
            elif (not use_lonlat) and kind == "xy":
                q = np.array([[bx, by]])
            else:
                # Coordinate kind mismatch; cannot reliably transform → use crude nearest by lon/lat if both exist,
                # else by x/y. If mismatch, try whichever exists in current network.
                if use_lonlat and kind == "xy":
                    # We only have xy for missing, but KDTree is lon/lat → fallback to closest by index 0
                    # (transparent about this in logs)
                    remap_report.append((c, None, None, "coord_kind_mismatch"))
                    # Pick the closest *by index* fallback: use network's geometric center
                    # (KDTree needs a point; we’ll use median lon/lat)
                    q = np.nanmedian(coords_now, axis=0)[None, :]
                elif (not use_lonlat) and kind == "lonlat":
                    remap_report.append((c, None, None, "coord_kind_mismatch"))
                    q = np.nanmedian(coords_now, axis=0)[None, :]
                else:
                    q = np.nanmedian(coords_now, axis=0)[None, :]
            idx = int(kdt_now.query(q)[1][0])
            nearest_bus = buses_now.index[idx]
            bus_assignments.append(nearest_bus)
            remap_report.append((c, bx, by, f"mapped_to:{nearest_bus}"))
        else:
            # No coordinates found anywhere → attach to globally nearest by network centroid
            idx = int(kdt_now.query(np.nanmedian(coords_now, axis=0)[None, :])[1][0])
            nearest_bus = buses_now.index[idx]
            bus_assignments.append(nearest_bus)
            remap_report.append((c, None, None, f"mapped_to:{nearest_bus};reason:no_coords"))

    if remap_report:
        logger.info("Load bus remapping performed (missing → nearest existing):")
        for r in remap_report:
            logger.info(f"  load '{r[0]}' → {r[3]} (src_coords: {r[1]}, {r[2]})")

    # --- Finally add Loads with reassigned buses
    network.madd("Load", load_cols, bus=bus_assignments, p_set=load_profile)

    # Peak (GW)
    peak_load = round(network.loads_t.p_set.T.sum().max() / 1000, 1)
    logger.info(f"Peak load in p_set time series: {peak_load} GW")

except Exception as e:
    logger.warning(f"Could not attach loads: {e}")



2025-11-11 11:01:00: Loading load profile: c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\processed\scaled_loads\load_base_2030_linear.csv
2025-11-11 11:01:00: 17 load buses not found; reassigning to nearest existing bus.
2025-11-11 11:01:00: Load bus remapping performed (missing → nearest existing):
2025-11-11 11:01:00:   load '109' → mapped_to:65 (src_coords: -78.7385, -2.9038)
2025-11-11 11:01:00:   load '122' → mapped_to:260 (src_coords: -78.7498, -1.7366)
2025-11-11 11:01:00:   load '134' → mapped_to:97 (src_coords: -78.7659, -3.8451)
2025-11-11 11:01:00:   load '135' → mapped_to:183 (src_coords: -79.9708, -3.5235)
2025-11-11 11:01:00:   load '168' → mapped_to:169 (src_coords: -80.5068, -0.8724)
2025-11-11 11:01:00:   load '172' → mapped_to:171 (src_coords: -80.1647, -0.857)
2025-11-11 11:01:00:   load '180' → mapped_to:179 (src_coords: -78.7958, 1.2621)
2025-11-11 11:01:00:   load '187' → mapped_to:176 (src_coords: -80.095, -0.8791)
2025-11-11 11:01:00:   load

In [4]:
# ======================================================================
#                      3) Power plants table (CSV)
# ======================================================================
ppl_csv = join(dirs["data/processed/generation"], "powerplants_all.csv")
assert os.path.isfile(ppl_csv), f"Power plants CSV not found: {ppl_csv}"
ppl_raw = pd.read_csv(ppl_csv, index_col=0)

# --- OPTIONAL (recommended): normalize using powerplantmatching
ppl = ppl_raw.copy()
try:
    import powerplantmatching as pm

    # Load PPM config (pypsa-earth config shipped in your repo)
    ppmatching = os.path.join(
        Path(os.getcwd()).parents[1], "pypsa-earth", "configs", "powerplantmatching_config.yaml"
    )
    print(ppmatching)
    config = pm.get_config(ppmatching)
    config["target_countries"] = ["EC"]  # Ecuador

    # These utilities are in pm.powerplant
    # 1) fill missing commissioning years
    ppl_f = ppl.powerplant.fill_missing_commissioning_years()
    # 2) convert to PyPSA-compatible column names (bus/generator naming conventions)
    ppl_p = ppl_f.powerplant.to_pypsa_names()

        # Drop all-empty columns that may be introduced by upstream merges
    ppl = ppl_p.dropna(axis=1, how="all")

    logger.info("Applied powerplantmatching normalization (fill years + to_pypsa_names).")
except Exception as e:
    logger.warning(f"powerplantmatching step skipped (using raw CSV): {e}")
    ppl = ppl_raw.copy()


ppl.columns

# ---- Normalize capacities and carriers (ES->EN) on the (possibly PPM-normalized) table
def _normalize_text(s):
    import unicodedata, re
    if pd.isna(s):
        return None
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"\s+", " ", s)
    return s

# capacity as float (handles "1'500,00")
if "p_nom" in ppl.columns:
    ppl["p_nom"] = (
        ppl["p_nom"].astype(str).str.replace("'", "", regex=False).str.replace(",", ".", regex=False)
    )
    ppl["p_nom"] = pd.to_numeric(ppl["p_nom"], errors="coerce").fillna(0.0)
else:
    logger.warning("Column 'p_nom' not found in plants table after normalization.")

# map carriers
if "carrier" in ppl.columns:
    ppl["carrier_norm"] = ppl["carrier"].apply(_normalize_text)
    carrier_map_es2en = {
        "hidraulica": "hydro",
        "hidroelectrico": "hydro",
        "termica": "oil",            # general fallback, could also be "natural gas" or "coal" depending on subtype
        "termoelectrico": "oil",
        "biomasa": "biomass",
        "fotovoltaica": "solar",
        "solar": "solar",
        "eolica": "onwind",
        "eolico": "onwind",
        "biogas": "biomass",         # or "biogas" if you keep it distinct later
        "ernc": "onwind",            # “Energía Renovable No Convencional”, usually wind/solar, adjust if needed
        "nuclear": "nuclear",
        "geotermica": "geothermal",
        "carbón": "coal",
        "carbon": "coal",
        "lignito": "lignite",
        "gas natural": "natural gas",
        "gas": "natural gas",
        "phs": "PHS",
        "pasada": "ror",             # hydro run-of-river
        "embalse": "hydro",          # reservoir-type hydro
        "offshore": "offwind-ac",    # adjust if you know DC vs AC
        "offshore_dc": "offwind-dc",
    }
    ppl["carrier"] = ppl["carrier_norm"].map(carrier_map_es2en).fillna(ppl["carrier_norm"])
else:
    logger.warning("Column 'carrier' not found in plants table after normalization.")

# ---- Filter: SNI, by year and minimum size
FILTERING_POWER = 10.0   # kW threshold
DATE_IN = 2017
if {"component", "DateIn", "p_nom"}.issubset(ppl.columns):
    ppl_connected = ppl[ppl["component"] == "S.N.I."]
    ppl_connected_current = ppl_connected[ppl_connected["DateIn"] <= DATE_IN]
    ppl_filtered = ppl_connected_current[ppl_connected_current["p_nom"] >= FILTERING_POWER].copy()
else:
    logger.warning("Required columns for filtering missing; skipping filter.")
    ppl_filtered = ppl.copy()

# ---- Totals
def safe_sum(series):
    try: return float(series.sum())
    except: return np.nan

total_capacity = safe_sum(ppl.get("p_nom", pd.Series(dtype=float)))
total_connected = safe_sum(ppl_connected.get("p_nom", pd.Series(dtype=float))) if 'ppl_connected' in locals() else np.nan
total_connected_current = safe_sum(ppl_connected_current.get("p_nom", pd.Series(dtype=float))) if 'ppl_connected_current' in locals() else np.nan
total_filtered = safe_sum(ppl_filtered.get("p_nom", pd.Series(dtype=float)))
logger.info(
    "Capacities [kW]\n"
    f"  total:                {total_capacity:,.0f}\n"
    f"  connected (SNI):      {total_connected:,.0f}\n"
    f"  connected <= {DATE_IN}: {total_connected_current:,.0f}\n"
    f"  filtered (>= {FILTERING_POWER:.0f} kW): {total_filtered:,.0f}"
)


c:\Repositories\Repos\pypsa-earth-project\pypsa-earth\configs\powerplantmatching_config.yaml
2025-11-11 11:01:00: Applied powerplantmatching normalization (fill years + to_pypsa_names).
2025-11-11 11:01:00: Capacities [kW]
  total:                16,076
  connected (SNI):      15,070
  connected <= 2017: 7,177
  filtered (>= 10 kW): 7,022


In [5]:
# 4) Attach filtered plants to nearest LV substation

from scipy.spatial import cKDTree as KDTree

def attach_to_nearest_lv_bus(net: pypsa.Network, plants_df: pd.DataFrame) -> pd.DataFrame:
    """Attach plants to nearest bus among those marked substation_lv."""
    if "substation_lv" not in net.buses.columns:
        raise ValueError("network.buses must contain a boolean 'substation_lv' column")

    if not {"lon", "lat"}.issubset(plants_df.columns):
        raise ValueError("plants_df must contain 'lon' and 'lat' columns")

    sub_idx = net.buses.query("substation_lv").index
    if len(sub_idx) == 0:
        raise ValueError("No LV substations found (substation_lv == True).")

    kdt = KDTree(net.buses.loc[sub_idx, ["x", "y"]].values)

    # Query nearest bus for each plant lon/lat -> need plant x/y; network.buses stores x,y in projected units.
    # If network.buses has lon/lat columns, prefer those:
    if {"lon", "lat"}.issubset(net.buses.columns):
        # Build a lon/lat->index KDTree as fallback using lon/lat and not x/y
        # But your buses KDTree above already used x/y; keep consistent by translating plants to bus CRS when available.
        pass

    # Best-effort: assume bus x/y ~ proj coords corresponding to lon/lat mapping already in network
    # If network carries bus lon/lat, we project plants into the same 'x/y' via a crude nearest in lon/lat space:
    # For robustness (and speed), use lon/lat KDTree when bus lon/lat exist:
    use_lonlat_tree = {"lon", "lat"}.issubset(net.buses.columns)
    if use_lonlat_tree:
        kdt_ll = KDTree(net.buses.loc[sub_idx, ["lon", "lat"]].values)
        tree_i = kdt_ll.query(plants_df.loc[:, ["lon", "lat"]].values)[1]
    else:
        # fallback to x/y KDTree (will work if plant lon/lat columns are actually x/y already)
        tree_i = kdt.query(plants_df.loc[:, ["lon", "lat"]].values)[1]

    attached = plants_df.copy()
    attached["bus"] = sub_idx.append(pd.Index([np.nan]))[tree_i].astype(str)
    missing = ~attached["bus"].isin(net.buses.index)
    if missing.any():
        logger.warning(f"Found {missing.sum()} plants with non-existing bus assignments.")
    return attached

try:
    ppl_attached = attach_to_nearest_lv_bus(network, ppl_filtered)
except Exception as e:
    logger.warning(f"Could not attach plants to LV buses: {e}")
    ppl_attached = ppl_filtered.copy()
    ppl_attached["bus"] = np.nan


In [6]:
# 5) Safely add generators to the network

def add_generators_safe(net: pypsa.Network, pp_df: pd.DataFrame):
    needed = ["bus", "carrier", "p_nom"]
    if not set(needed).issubset(pp_df.columns):
        raise ValueError(f"Missing columns in plant df. Need at least: {needed}")

    # PyPSA expects per-generator rows. We’ll use index as names.
    added = 0
    for i in pp_df.index:
        row = pp_df.loc[[i]]
        try:
            
            net.madd("Generator",
                     row.index,
                     bus=row["bus"].values,
                     carrier=row["carrier"].values,
                     p_nom=row["p_nom"].values)
            added += 1
        except Exception as e:
            logger.warning(f"  ⚠ could not add {i}: {e}")
    logger.info(f"Generators added: {added}")

try:
    add_generators_safe(network, ppl_attached)
except Exception as e:
    logger.warning(f"Adding generators failed: {e}")


2025-11-11 11:01:01: Generators added: 64


In [7]:
# 6) Technology costs merge
try:
    cost_filename = join(dirs["data/raw/generation"], "technology_cost.csv")
    costs = pd.read_csv(cost_filename, index_col=0, comment="#")
    # Expecting index to be carrier names; columns e.g. marginal_cost, capital_cost, etc.
    # Remove any pre-existing columns to avoid duplicate merge keys
    for c in ["marginal_cost", "capital_cost"]:
        if c in network.generators.columns:
            network.generators.drop(columns=[c], inplace=True)

    network.generators = pd.merge(
        network.generators,
        costs,
        left_on="carrier",
        right_index=True,
        how="left",
        suffixes=("", "_cost"),
    )
    logger.info("Merged tech costs into network.generators")
except Exception as e:
    logger.warning(f"Could not merge technology costs: {e}")


2025-11-11 11:01:01: Merged tech costs into network.generators


Assertion that the buses RHS and LHS of the equation are balanced

In [8]:
import pandas as pd
import networkx as nx

n = network  # just a shorter alias

# 1) Check you actually have snapshots and they match your time series
print("Snapshots:", n.snapshots[:3], "... total:", len(n.snapshots))

# 2) Buses that carry any non-zero RHS (net fixed injections/withdrawals)
rhs_load = (n.loads_t.p_set.groupby(n.loads.bus, axis=1).sum()
            if not n.loads_t.p_set.empty else pd.DataFrame(index=n.snapshots))
rhs_other = 0
# add other fixed RHS terms if you use them, e.g. fixed generator p_set, shunts, etc.

rhs_by_bus = rhs_load.sum(axis=0)  # sum over time
rhs_nonzero = rhs_by_bus[abs(rhs_by_bus) > 1e-6]

print("Buses with non-zero RHS count:", rhs_nonzero.shape[0])

# 3) Buses that have any LHS variables attached (anything that can balance)
lhs_buses = set()

# Generators with capacity (fixed or extendable)
if len(n.generators):
    cap = n.generators.p_nom.copy()
    cap[n.generators.p_nom_extendable.fillna(False)] = cap.where(~n.generators.p_nom_extendable, 1.0)
    lhs_buses |= set(n.generators.bus[cap > 0])

# StorageUnits / Stores
if len(n.storage_units):
    cap = n.storage_units.p_nom.copy()
    cap[n.storage_units.p_nom_extendable.fillna(False)] = cap.where(~n.storage_units.p_nom_extendable, 1.0)
    lhs_buses |= set(n.storage_units.bus[cap > 0])

if len(n.stores):
    cap = n.stores.e_nom.copy()
    cap[n.stores.e_nom_extendable.fillna(False)] = cap.where(~n.stores.e_nom_extendable, 1.0)
    lhs_buses |= set(n.stores.bus[cap > 0])

# Lines / Transformers (any positive rating yields flow variables)
if len(n.lines):
    lhs_buses |= set(n.lines.bus0[n.lines.s_nom > 0])
    lhs_buses |= set(n.lines.bus1[n.lines.s_nom > 0])

if len(n.transformers):
    lhs_buses |= set(n.transformers.bus0[n.transformers.s_nom > 0])
    lhs_buses |= set(n.transformers.bus1[n.transformers.s_nom > 0])

# Links (DC/HVDC links also provide variables)
if len(n.links):
    lhs_buses |= set(n.links.bus0[n.links.p_nom > 0])
    lhs_buses |= set(n.links.bus1[n.links.p_nom > 0])

problem_buses = [b for b in rhs_nonzero.index if b not in lhs_buses]
print("Buses with RHS≠0 but no LHS variables:", problem_buses)

# 4) Also check for islands without supply
G = n.graph()
islands = list(nx.connected_components(G))
print(f"Connected components: {len(islands)}")


Snapshots: DatetimeIndex(['2013-01-01 00:00:00', '2013-01-01 01:00:00',
               '2013-01-01 02:00:00'],
              dtype='datetime64[ns]', name='snapshot', freq=None) ... total: 8760
Buses with non-zero RHS count: 235
Buses with RHS≠0 but no LHS variables: ['90']
Connected components: 24


In [9]:
b = "90"  # o 90, según tu índice; abajo normalizamos a str

def _mask_bus(df, col="bus", bus=b):
    if df.empty or col not in df.columns: 
        return df.iloc[0:0]
    return df[df[col].astype(str) == str(bus)]

loads_90        = _mask_bus(network.loads, "bus")
gens_90         = _mask_bus(network.generators, "bus")
stores_90       = _mask_bus(network.stores, "bus")
sus_90          = _mask_bus(network.storage_units, "bus")
shunts_90       = _mask_bus(network.shunt_impedances, "bus") if hasattr(network, "shunt_impedances") else network.buses.iloc[0:0]

lines_90_0      = _mask_bus(network.lines, "bus0")
lines_90_1      = _mask_bus(network.lines, "bus1")
trafos_90_0     = _mask_bus(network.transformers, "bus0")
trafos_90_1     = _mask_bus(network.transformers, "bus1")
links_90_0      = _mask_bus(network.links, "bus0")
links_90_1      = _mask_bus(network.links, "bus1")

print("Loads @90:\n", loads_90)
print("Generators @90:\n", gens_90)
print("Stores @90:\n", stores_90)
print("StorageUnits @90:\n", sus_90)
print("Shunts @90:\n", shunts_90)

print("Lines touching 90 (as bus0):\n", lines_90_0.index.tolist())
print("Lines touching 90 (as bus1):\n", lines_90_1.index.tolist())
print("Transformers touching 90 (bus0):\n", trafos_90_0.index.tolist())
print("Transformers touching 90 (bus1):\n", trafos_90_1.index.tolist())
print("Links touching 90 (bus0):\n", links_90_0.index.tolist())
print("Links touching 90 (bus1):\n", links_90_1.index.tolist())


Loads @90:
      bus carrier type  p_set  q_set  sign
Load                                     
90    90                 0.0    0.0  -1.0
Generators @90:
 Empty DataFrame
Columns: [bus, carrier, p_nom, control, type, p_nom_mod, p_nom_extendable, p_nom_min, p_nom_max, p_min_pu, p_max_pu, p_set, q_set, sign, marginal_cost_quadratic, build_year, lifetime, efficiency, committable, start_up_cost, shut_down_cost, stand_by_cost, min_up_time, min_down_time, up_time_before, down_time_before, ramp_limit_up, ramp_limit_down, ramp_limit_start_up, ramp_limit_shut_down, weight, p_nom_opt, marginal_cost, capital_cost, notes]
Index: []

[0 rows x 35 columns]
Stores @90:
 Empty DataFrame
Columns: [bus, type, carrier, e_nom, e_nom_mod, e_nom_extendable, e_nom_min, e_nom_max, e_min_pu, e_max_pu, e_initial, e_initial_per_period, e_cyclic, e_cyclic_per_period, p_set, q_set, sign, marginal_cost, marginal_cost_quadratic, marginal_cost_storage, capital_cost, standing_loss, build_year, lifetime, e_nom_opt]
Ind

In [10]:
import networkx as nx
for comp in nx.connected_components(network.graph()):
    b = next(iter(comp))
    print(comp, b)
    if f"slack_{b}" not in network.generators.index:
        print("here it would be added" + 
              f"""
    #    network.add("Generator", f"slack_{b}", bus=b, p_nom=1e6, marginal_cost=1e6)
    """)


{'224', '10', '44', '80', '267', '285', '115', '167', '61', '106', '24', '269', '31', '185', '129', '86', '33', '120', '37', '217', '81', '82', '20', '311', '137', '160', '273', '239', '233', '306', '272', '195', '97', '30', '83', '251', '84', '21', '92', '72', '198', '266', '245', '228', '48', '220', '8', '234', '247', '42', '102', '294', '9', '139', '2', '216', '25', '194', '60', '87', '56', '252', '67', '104', '40', '146', '188', '79', '197', '34', '142', '230', '57', '221', '11', '23', '268', '282', '7', '257', '141', '6', '27', '126', '47', '35', '301', '118', '236', '260', '77', '32', '256', '166', '175', '103', '112', '145', '132', '144', '138', '265', '204', '229', '1', '302', '157', '250', '19', '173', '296', '205', '161', '231', '222', '312', '303', '152', '154', '232', '38', '248', '14', '281', '290', '75', '93', '52', '123', '16', '147', '304', '215', '28', '43', '55', '121', '3', '111', '249', '58', '270', '73', '261', '17', '100', '298', '140', '263', '29', '227', '305', 

In [11]:

# manually remove bus 90, just to test 
bus_id = "90"

# 1. Remove any loads connected to that bus

loads_to_remove = network.loads.index[network.loads.bus.astype(str) == str(bus_id)]
for load_id in loads_to_remove:
    network.remove("Load", load_id)
print(f"Removed loads connected to bus {bus_id}: {list(loads_to_remove)}")

# 2. Remove the bus itself
if bus_id in network.buses.index.astype(str):
    network.remove("Bus", bus_id)
    print(f"Removed bus {bus_id}")
else:
    print(f"Bus {bus_id} not found in network.")



Removed loads connected to bus 90: ['90']
Removed bus 90


In [22]:
network.carriers = pd.read_csv(
    join(dirs["data/raw/generation"], "carriers.csv"), index_col=0
).reset_index()

network.carriers

network.generators.carrier.unique()

network.generators.to_csv("carriers_gen.csv")

In [13]:
network.lines['x'] = 0.1
network.lines['r'] = 0.01
solver = "highs"
# Select first 4 snapshots
snapshots_subset = network.snapshots[:4]

# Optimize only over those 4 snapshots
network.optimize(snapshots=snapshots_subset, solver_name=solver)

2025-11-11 11:01:01: The following transformers have zero r, which could break the linear load flow:
Index(['transf_0_0', 'transf_1_0', 'transf_2_0', 'transf_3_0', 'transf_3_1',
       'transf_4_0', 'transf_5_0', 'transf_6_0', 'transf_7_0', 'transf_7_1',
       'transf_8_0', 'transf_9_0', 'transf_9_1', 'transf_10_0', 'transf_10_1',
       'transf_10_2', 'transf_11_0', 'transf_12_0', 'transf_13_0',
       'transf_14_0', 'transf_15_0', 'transf_16_0', 'transf_17_0',
       'transf_18_0', 'transf_18_1', 'transf_19_0', 'transf_20_0',
       'transf_20_1', 'transf_21_0', 'transf_23_0', 'transf_24_0',
       'transf_26_0', 'transf_27_0', 'transf_28_0', 'transf_29_0',
       'transf_32_0', 'transf_32_1', 'transf_33_0', 'transf_34_0',
       'transf_35_0', 'transf_36_0', 'transf_36_1', 'transf_37_0',
       'transf_40_0', 'transf_42_0', 'transf_43_0', 'transf_45_0',
       'transf_45_1', 'transf_46_0', 'transf_47_0', 'transf_48_0',
       'transf_48_1', 'transf_49_0', 'transf_49_1', 'transf_50_

2025-11-11 11:01:01: The following buses have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '303', '304', '305', '306', '307', '308', '309', '310', '311', '312'],
      dtype='object', name='Bus', length=295)
2025-11-11 11:01:01: The following lines have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '237', '238', '239', '240', '241', '242', '243', '244', '245', '246'],
      dtype='object', name='Line', length=247)
2025-11-11 11:01:01: The following generators have carriers which are not defined:
Index(['Abanico', 'Agoyan', 'Alao', 'Alvaro_Tinajero', 'Anibal_Santos_(Gas)',
       'Anibal_Santos_(Vapor)', 'Baba', 'Calope', 'Catamayo',
       'Coca_Codo_Sinclair', 'Cumbaya', 'Delsitanisagua_0', 'Due',
       'Ecoelectric', 'Ecudos_A-G', 'El_Descanso', 'Enrique_Garcia',
       'Esmeraldas_I', 'Esmeraldas_II', 'Generoca', 'Gonzalo_Zevallos_(Gas)',
       'Gonzalo_Zevall

('warning', 'infeasible')

In [14]:
network.buses.substation_lv = True

In [15]:
network.carriers

#attempt to solve for 1 month w/o clusering



,Carrier,co2_emissions,color,nice_name,max_growth,max_relative_growth
0,coal,0.354,#707070,Coal,inf,0.0
1,natural gas,0.187,#d35050,Natural Gas,inf,0.0
2,nuclear,0.000,#ff9000,Nuclear,inf,0.0
3,oil,0.248,#262626,Oil,inf,0.0
4,lignite,0.334,#9e5a01,Lignite,inf,0.0
5,geothermal,0.026,#ba91b1,Geothermal,inf,0.0
6,biomass,0.000,#0c6013,Biomass,inf,0.0
7,hydro,0.000,#08ad97,Reservoir & Dam,inf,0.0
8,offwind-ac,0.000,#6895dd,Offshore Wind (AC),inf,0.0
9,offwind-dc,0.000,#74c6f2,Offshore Wind (DC),inf,0.0


In [16]:
# 7) GADM regions and bus mapping (with nearest-region fallback)
COUNTRY_ISO3 = "ECU"
GADM_VERSION = "4.1"
GADM_FILE_STEM = f"gadm41_{COUNTRY_ISO3}"
GADM_URL = f"https://geodata.ucdavis.edu/gadm/gadm{GADM_VERSION}/gpkg/{GADM_FILE_STEM}.gpkg"

def ensure_gadm_file(gadm_path: str) -> str:
    gpkg_path = os.path.join(gadm_path, f"{GADM_FILE_STEM}.gpkg")
    gpkg_dir = Path(gpkg_path).parent
    if not Path(gpkg_path).is_file():
        gpkg_dir.mkdir(parents=True, exist_ok=True)
        import requests, shutil as _shutil
        logger.info(f"Downloading GADM: {GADM_URL}")
        resp = requests.get(GADM_URL, stream=True, timeout=300)
        resp.raise_for_status()
        with open(gpkg_path, "wb") as f:
            _shutil.copyfileobj(resp.raw, f)
    assert Path(gpkg_path).is_file(), f"GADM not found or failed to download: {gpkg_path}"
    return gpkg_path

try:
    gadm_file = ensure_gadm_file(dirs["data/raw/gadm"])
    shapefile = gpd.read_file(gadm_file, layer="ADM_ADM_1")
except Exception as e:
    shapefile = None
    logger.warning(f"Could not load GADM: {e}")

def get_node_by_region(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Ensure EPSG:4326 and add centroid x/y for convenience."""
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    gdf = gdf.copy()
    gdf["x"] = gdf.geometry.centroid.x
    gdf["y"] = gdf.geometry.centroid.y
    return gdf

def _haversine_km(lon1, lat1, lon2, lat2):
    """Approximate distance (km) on a sphere."""
    import math
    R = 6371.0
    dlon = math.radians(lon2 - lon1)
    dlat = math.radians(lat2 - lat1)
    a = (math.sin(dlat/2)**2 +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2)
    return 2 * R * math.asin(math.sqrt(a))

def attach_node_to_region_or_nearest(gdf: gpd.GeoDataFrame, longitude: float, latitude: float):
    """
    Try strict point-in-polygon. If no hit, assign the nearest polygon by centroid,
    returning (region_name, region_centroid_x, region_centroid_y, distance_km, used_nearest: bool).
    """
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    pt = Point(float(longitude), float(latitude))

    # 1) strict containment
    hit = gdf[gdf.contains(pt)]
    if not hit.empty:
        row = hit.iloc[0]
        return row.get("NAME_1", "NA"), row.get("x", np.nan), row.get("y", np.nan), 0.0, False

    # 2) nearest by centroid (fast and robust; good enough for admin-1)
    # If an sindex exists, you could use it; here we just compute argmin over centroids:
    dx = (gdf["x"] - float(longitude))
    dy = (gdf["y"] - float(latitude))
    # pick row with minimal haversine distance
    d_km = _haversine_km(float(longitude), float(latitude), gdf["x"], gdf["y"])
    # d_km is a Series due to vectorized haversine
    nearest_idx = d_km.astype(float).idxmin()
    row = gdf.loc[nearest_idx]
    return row.get("NAME_1", "NA"), row.get("x", np.nan), row.get("y", np.nan), float(d_km.loc[nearest_idx]), True

def attach_region_to_buses(gdf: gpd.GeoDataFrame, buses: pd.DataFrame) -> pd.DataFrame:
    """
    Attach each bus to a region. If the bus is on a border or outside polygons,
    assign the nearest region and LOG it with lon/lat and distance.
    """
    buses = buses.copy()
    buses["region"] = "NA"
    buses["region_x"] = np.nan
    buses["region_y"] = np.nan

    if gdf is None or len(gdf) == 0:
        logger.warning("No GADM dataframe available; skipping region attachment.")
        return buses

    # Ensure we have lon/lat
    if not {"lon", "lat"}.issubset(buses.columns):
        logger.warning("Buses lack lon/lat; cannot attach regions.")
        return buses

    # Iterate and attach
    for i in buses.index:
        try:
            lon = float(buses.at[i, "lon"])
            lat = float(buses.at[i, "lat"])
            if not np.isfinite(lon) or not np.isfinite(lat):
                continue
            r, rx, ry, d_km, used_nearest = attach_node_to_region_or_nearest(gdf, lon, lat)
            buses.at[i, "region"] = r
            buses.at[i, "region_x"] = rx
            buses.at[i, "region_y"] = ry
            if used_nearest:
                logger.warning(
                    f"Bus '{i}' was outside all polygons; assigned to nearest region='{r}' "
                    f"(bus lon/lat=({lon:.6f},{lat:.6f}), ~{d_km:.1f} km to region centroid)."
                )
        except Exception as ex:
            # keep as NA and continue
            logger.warning(f"Region attach failed for bus '{i}': {ex}")
            continue
    return buses

# Build busmap and (optionally) cluster
busmap = None
if shapefile is not None:
    region_node = get_node_by_region(shapefile)
    try:
        buses_with_region = attach_region_to_buses(region_node, network.buses)
        # Ensure busmap index matches network.buses.index exactly
        buses_with_region = buses_with_region.loc[network.buses.index]
        busmap = buses_with_region["region"]
        busmap.to_csv("bus_mapped.csv")
        logger.info("Bus-to-region mapping written to bus_mapped.csv")
    except Exception as e:
        logger.warning(f"Could not attach regions to buses: {e}")

# --- Enforce identical bus coordinates within each region (Option A) ---
# We will overwrite bus lon/lat (and x/y if present) with the region centroid,
# so that all buses belonging to the same region have identical coordinates.
# This satisfies PyPSA's "attributes must agree within cluster" requirement.

if shapefile is not None and busmap is not None and busmap.notna().all():
    # 1) Build region centroids (EPSG:4326) from the already-prepared region_node
    #    region_node has columns: NAME_1, x, y (where x=lon, y=lat)
    if not {"NAME_1", "x", "y"}.issubset(region_node.columns):
        logger.warning("Region node lacks NAME_1/x/y; cannot enforce centroid coordinates.")
    else:
        # Align busmap with network and cast indices consistently
        if not busmap.index.equals(network.buses.index):
            busmap = busmap.reindex(network.buses.index)
            logger.info("Aligned busmap index to network.buses.index.")

        # 2) Basic sanity checks before mutation
        n_buses = len(network.buses)
        n_unassigned = busmap.isna().sum()
        if n_unassigned:
            logger.warning(f"{n_unassigned} buses lack region labels; these will remain unchanged.")
        logger.info(f"Unique regions in busmap: {busmap.dropna().nunique()} (over {n_buses} buses)")

        # 3) Map region label -> centroid lon/lat
        centroids = region_node.set_index("NAME_1")[["x", "y"]].rename(columns={"x": "cent_lon", "y": "cent_lat"})
        missing_regions = sorted(set(busmap.dropna().unique()) - set(centroids.index))
        if missing_regions:
            logger.warning(f"{len(missing_regions)} region labels not found in centroids: {missing_regions[:5]}...")

        # 4) Apply representative lon/lat per region to buses
        buses_mut = network.buses.copy()
        # Ensure lon/lat columns exist
        for col in ("lon", "lat"):
            if col not in buses_mut.columns:
                buses_mut[col] = np.nan

        # Join centroids onto each bus via busmap
        rep_coords = busmap.to_frame("region").join(centroids, how="left", on="region")

        # Count how many will be updated
        n_to_update = rep_coords["cent_lon"].notna().sum()
        logger.info(f"Setting representative lon/lat from region centroids for {n_to_update} buses.")

        # Overwrite lon/lat where centroid available
        buses_mut.loc[rep_coords["cent_lon"].notna(), "lon"] = rep_coords.loc[rep_coords["cent_lon"].notna(), "cent_lon"].values
        buses_mut.loc[rep_coords["cent_lat"].notna(), "lat"] = rep_coords.loc[rep_coords["cent_lat"].notna(), "cent_lat"].values

        # 5) Recompute x/y from lon/lat so everything is consistent (projected CRS)
        #    Use a metric CRS (Web Mercator EPSG:3857 is fine for consistency here).
        try:
            gtmp = gpd.GeoDataFrame(
                buses_mut,
                geometry=gpd.points_from_xy(buses_mut["lon"], buses_mut["lat"], crs="EPSG:4326")
            )
            gtmp_merc = gtmp.to_crs("EPSG:3857")
            # Ensure x/y exist and overwrite with projected coords
            buses_mut["x"] = gtmp_merc.geometry.x.values
            buses_mut["y"] = gtmp_merc.geometry.y.values
            # drop geometry to keep PyPSA table tidy
            buses_mut = pd.DataFrame(buses_mut).drop(columns=["geometry"], errors="ignore")
            logger.info("Recomputed bus x/y from lon/lat in EPSG:3857.")
        except Exception as ex:
            logger.warning(f"Could not recompute x/y from lon/lat: {ex}. Proceeding with lon/lat only.")

        # 6) Sanity checks after mutation
        #    a) Within each region, lon/lat should be uniform
        probe_regions = busmap.dropna().unique()[:5]  # sample a few for log brevity
        for r in probe_regions:
            idx = busmap[busmap == r].index
            if len(idx) > 1:
                lon_nunique = buses_mut.loc[idx, "lon"].nunique(dropna=False)
                lat_nunique = buses_mut.loc[idx, "lat"].nunique(dropna=False)
                if lon_nunique > 1 or lat_nunique > 1:
                    logger.warning(f"Region '{r}': lon unique={lon_nunique}, lat unique={lat_nunique} (expected 1).")

        #    b) No NaNs in lon/lat for clustered buses
        n_missing_coords = buses_mut.loc[busmap.notna(), ["lon", "lat"]].isna().any(axis=1).sum()
        if n_missing_coords:
            logger.warning(f"{n_missing_coords} clustered buses still have missing lon/lat after centroid assignment.")

        # 7) Write back to the network (mutating the active network)
        network.buses.loc[buses_mut.index, buses_mut.columns] = buses_mut

        # 8) Final note
        logger.info("Assigned representative coordinates per region to all clustered buses.")
else:
    logger.warning("Skipping coordinate unification: no shapefile or busmap with full labels.")

# --- Harmonize per-cluster voltage attributes (v_nom_raw and v_nom) ---
try:
    if shapefile is not None and busmap is not None and busmap.notna().all():
        # Align index just in case
        busmap = busmap.reindex(network.buses.index)

        buses_mut = network.buses.copy()

        # Ensure the columns exist and are numeric
        if "v_nom_raw" not in buses_mut.columns and "v_nom" in buses_mut.columns:
            buses_mut["v_nom_raw"] = buses_mut["v_nom"]
        if "v_nom_raw" not in buses_mut.columns:
            buses_mut["v_nom_raw"] = np.nan
        if "v_nom" not in buses_mut.columns:
            buses_mut["v_nom"] = buses_mut["v_nom_raw"]

        # Coerce to numeric
        buses_mut["v_nom_raw"] = pd.to_numeric(buses_mut["v_nom_raw"], errors="coerce")
        buses_mut["v_nom"]      = pd.to_numeric(buses_mut["v_nom"], errors="coerce")

        # Allowed Ecuador voltage levels (kV)
        allowed_levels = np.array([69.0, 138.0, 230.0, 500.0])

        def _snap_voltage_kv(values: pd.Series) -> pd.Series:
            """Snap numeric series to nearest allowed voltage level."""
            arr = pd.to_numeric(values, errors="coerce").values.astype(float)
            snapped = np.full_like(arr, np.nan, dtype=float)
            mask = np.isfinite(arr)
            if mask.any():
                # vectorized nearest snap
                diffs = np.abs(arr[mask, None] - allowed_levels[None, :])
                nearest_idx = np.argmin(diffs, axis=1)
                snapped[mask] = allowed_levels[nearest_idx]
            return pd.Series(snapped, index=values.index)

        # Precompute snapped variants (helps with messy inputs)
        snapped_raw = _snap_voltage_kv(buses_mut["v_nom_raw"])
        snapped_nom = _snap_voltage_kv(buses_mut["v_nom"])
        # Prefer raw if available; otherwise use v_nom
        snapped_pref = snapped_raw.where(snapped_raw.notna(), snapped_nom)

        changed_clusters = 0
        total_clusters = 0

        for region_label, idx in busmap.groupby(busmap).groups.items():
            total_clusters += 1
            # volts in this cluster (snapped)
            v_cluster = snapped_pref.loc[idx]
            # if already uniform (or all NaN), skip
            uniq = v_cluster.dropna().unique()
            if len(uniq) <= 1:
                # still write back snapping to keep internal consistency
                if len(uniq) == 1:
                    rep = float(uniq[0])
                    buses_mut.loc[idx, "v_nom_raw"] = rep
                    buses_mut.loc[idx, "v_nom"] = rep
                continue

            # Choose representative: mode (majority). If tie, choose higher kV.
            vc = v_cluster.value_counts(dropna=True)
            if vc.empty:
                # all NaN → skip; clustering won’t include these if busmap has NaN
                continue

            top = vc.max()
            candidates = vc[vc == top].index.astype(float)
            rep = float(np.max(candidates))  # tie → higher level

            # Apply to both fields so PyPSA sees consistent values
            buses_mut.loc[idx, "v_nom_raw"] = rep
            buses_mut.loc[idx, "v_nom"] = rep
            changed_clusters += 1

            logger.warning(
                f"Cluster '{region_label}': unified voltages to {rep:.0f} kV. "
                f"Counts (snapped): {vc.to_dict()}"
            )

        if changed_clusters:
            logger.info(
                f"Harmonized voltage in {changed_clusters}/{total_clusters} region clusters "
                f"(fields: v_nom_raw, v_nom)."
            )

        # Write back
        network.buses.loc[buses_mut.index, ["v_nom_raw", "v_nom"]] = buses_mut[["v_nom_raw", "v_nom"]]

    else:
        logger.warning("Skipping voltage harmonization: shapefile/busmap not ready.")
except Exception as ex:
    logger.warning(f"Failed to harmonize per-cluster voltages: {ex}")


# Optional: clustering when busmap is available
try:
    if busmap is not None and busmap.notna().all():
        from pypsa.clustering.spatial import get_clustering_from_busmap
        clustered = get_clustering_from_busmap(network, busmap).network
        clustered.generators.to_csv("clustered_gens.csv")
        logger.info("Clustered network generators exported to clustered_gens.csv")
    else:
        logger.warning("Busmap missing or contains NaNs; skipping clustering.")
except Exception as e:
    logger.warning(f"Clustering failed: {e}")


2025-11-11 11:01:03: Bus-to-region mapping written to bus_mapped.csv
2025-11-11 11:01:03: Unique regions in busmap: 23 (over 295 buses)
2025-11-11 11:01:03: Setting representative lon/lat from region centroids for 295 buses.
2025-11-11 11:01:03: Recomputed bus x/y from lon/lat in EPSG:3857.
2025-11-11 11:01:03: Assigned representative coordinates per region to all clustered buses.
2025-11-11 11:01:03: Cluster 'Azuay': unified voltages to 230 kV. Counts (snapped): {230.0: 4, 69.0: 3, 138.0: 2}
2025-11-11 11:01:03: Cluster 'Bolivar': unified voltages to 69 kV. Counts (snapped): {69.0: 5, 500.0: 1}
2025-11-11 11:01:03: Cluster 'Carchi': unified voltages to 69 kV. Counts (snapped): {69.0: 5, 138.0: 1}
2025-11-11 11:01:03: Cluster 'Cañar': unified voltages to 69 kV. Counts (snapped): {69.0: 6, 230.0: 4}
2025-11-11 11:01:03: Cluster 'Chimborazo': unified voltages to 69 kV. Counts (snapped): {69.0: 6, 230.0: 1}
2025-11-11 11:01:03: Cluster 'Cotopaxi': unified voltages to 69 kV. Counts (snappe

In [17]:
clustered

NameError: name 'clustered' is not defined